In [1]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet")
Alpha_Name = "AlphaNet_Original_Input_12.14"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "5d_ret"

In [3]:
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/feature_platform")
alpha_list = list(FT.features_in_path.keys())[:]
alpha_list = ['ewmCorr(high,low,7)','ewmCov(open,high,7)']
print(alpha_list)
start_date = '2015-01-01'
end_date = '2022-01-01'
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   4%|███▋                                                                                     | 140/3410 [00:00<00:04, 804.13it/s]

['ewmCorr(high,low,7)', 'ewmCov(open,high,7)']


Loading Feature...:  86%|███████████████████████████████████████████████████████████████████████████▌            | 2927/3410 [00:05<00:01, 439.46it/s]

(                            ewmCorr(high,low,7)  ewmCov(open,high,7)
 timestamp           ticker                                          
 2016-01-05 09:30:00 002269             0.261552            -0.020979
                     000629             0.261552            -0.020978
                     603022             0.261552            -0.020970
                     600617             0.261552            -0.020860
                     002684             0.261552            -0.020868
 ...                                         ...                  ...
 2021-07-01 09:30:00 300230            -4.442220            -0.037525
                     002319            -4.504204            -0.036942
                     603286            -4.533666            -0.037445
                     000796            -4.536809            -0.033297
                     002707            -4.536859            -0.020746
 
 [4444115 rows x 2 columns],
 None)

In [4]:
data = FT.feature_data
data

ewmCorr(high,low,7)  ewmCov(open,high,7)
timestamp           ticker                                          
2016-01-05 09:30:00 002269             0.261552            -0.020979
                    000629             0.261552            -0.020978
                    603022             0.261552            -0.020970
                    600617             0.261552            -0.020860
                    002684             0.261552            -0.020868
...                                         ...                  ...
2021-07-01 09:30:00 300230            -4.442220            -0.037525
                    002319            -4.504204            -0.036942
                    603286            -4.533666            -0.037445
                    000796            -4.536809            -0.033297
                    002707            -4.536859            -0.020746

[4444115 rows x 2 columns]

In [5]:
import AlphaNet.DataProcessing.Norm as Norm
output_path = r'/home/wuwenjun/feature_platform/ti0/wuwenjun'
Norm.time_series_norm(data=data,rolling=30,output_path = output_path)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [8]:
grouped = data.groupby("timestamp")

ewmCorr(high,low,7)  ewmCov(open,high,7)
timestamp           ticker                                          
2016-01-05 09:30:00 002269             0.261552            -0.020979
                    000629             0.261552            -0.020978
                    603022             0.261552            -0.020970
                    600617             0.261552            -0.020860
                    002684             0.261552            -0.020868
...                                         ...                  ...
2021-07-01 09:30:00 300230            -4.442220            -0.037525
                    002319            -4.504204            -0.036942
                    603286            -4.533666            -0.037445
                    000796            -4.536809            -0.033297
                    002707            -4.536859            -0.020746

[4444115 rows x 2 columns]

In [12]:
def section_norm(data=data,rolling=30,output_path = output_path):
    grouped = data.groupby("timestamp")
    df_list = []
    for ticker , df in tqdm(grouped):
        for column in df.columns:
            df[column] = norm.rationalize_s(df[column], weight=None, sigma=1, fill=True, fillvalue='mean')
            df[column] = norm.normalize_s(df[column], weight=None, sigma=1, fill=True, fillvalue='mean')
        df_list.append(df)
    df_list = pd.concat(df_list,axis=0)
    df_list.columns = [str(column)+"_section_norm" for column in df_list.columns]
df_list


ewmCorr(high,low,7)  ewmCov(open,high,7)
timestamp           ticker                                          
2016-01-05 09:30:00 002269             0.267339            -0.020979
                    000629             0.267339            -0.020978
                    603022             0.267339            -0.020970
                    600617             0.267339            -0.020860
                    002684             0.267339            -0.020868
...                                         ...                  ...
2021-07-01 09:30:00 300230            -1.477739            -0.037525
                    002319            -1.477739            -0.036942
                    603286            -1.477739            -0.037445
                    000796            -1.477739            -0.033297
                    002707            -1.477739            -0.020746

[4444115 rows x 2 columns]

In [13]:
grouped = data.groupby("timestamp")
df_list = []
for ticker , df in tqdm(grouped):
    for column in df.columns:
        df[column] = norm.rationalize_s(df[column], weight=None, sigma=1, fill=True, fillvalue='mean')
        df[column] = norm.normalize_s(df[column], weight=None, sigma=1, fill=True, fillvalue='mean')
    df_list.append(df)
df_list = pd.concat(df_list,axis=0)
df_list.columns = [str(column)+"_section_norm" for column in df_list.columns]

100%|██████████| 1336/1336 [00:48<00:00, 27.39it/s]


In [14]:
df_list

ewmCorr(high,low,7)_section_norm  \
timestamp           ticker                                     
2016-01-05 09:30:00 002269                          0.267339   
                    000629                          0.267339   
                    603022                          0.267339   
                    600617                          0.267339   
                    002684                          0.267339   
...                                                      ...   
2021-07-01 09:30:00 300230                         -1.477739   
                    002319                         -1.477739   
                    603286                         -1.477739   
                    000796                         -1.477739   
                    002707                         -1.477739   

                            ewmCov(open,high,7)_section_norm  
timestamp           ticker                                    
2016-01-05 09:30:00 002269                         -0.059295  
                    000629                         -0.058484  
                    603022                         -0.053519  
                    600617                          0.018495  
                    002684                          0.013525  
...                                                      ...  
2021-07-01 09:30:00 300230                         -0.243731  
                    002319                         -0.204552  
                    603286                         -0.238390  
                    000796                          0.040469  
                    002707                          0.884242  

[4444115 rows x 2 columns]